In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Create flag parameter

In [0]:
dbutils.widgets.text("incremental_flag", "0")

In [0]:
incremental_flag = dbutils.widgets.get("incremental_flag")
print(incremental_flag)

0


# Creating dimension model

### Fetch Relative Columns

In [0]:
df_source = spark.sql('''
                      select distinct(Model_ID), model_category from parquet.`abfss://silver@carrishitdatalake.dfs.core.windows.net/carSales`
                      ''')
df_source.display()

Model_ID,model_category
Mah-M167,Mah
Che-M47,Che
Toy-M205,Toy
BMW-M249,BMW
Mer-M122,Mer
Hon-M215,Hon
Nis-M82,Nis
Toy-M206,Toy
Mar-M139,Mar
Ren-M207,Ren


### dim_model Sink - Inital and Incremental

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.dim_model"):
        df_sink = spark.sql('''
                select dim_model_key, Model_ID, model_category from delta.`abfss://gold@carrishitdatalake.dfs.core.windows.net/dim_model`
                ''')
        pass
else:
        df_sink = spark.sql('''
                select 1 as dim_model_key, Model_ID, model_category from parquet.`abfss://silver@carrishitdatalake.dfs.core.windows.net/carSales` where 1 = 0
                ''')

### Filtering New Records and Old Records

In [0]:
df_filter = df_source.join(df_sink, df_source.Model_ID == df_sink.Model_ID, 'left').select(df_source.Model_ID, df_source.model_category, df_sink.dim_model_key)
df_filter_new = df_filter.filter(df_filter.dim_model_key.isNull())
df_filter_old = df_filter.filter(df_filter.dim_model_key.isNotNull())


creating Surrogate Key

In [0]:
if incremental_flag == "0":
    max_value = 1
else:
    max_value = df_filter_old.select(max(df_filter_old.dim_model_key)).collect()[0][0] + 1

print(max_value)
df_filter_new = df_filter_new.withColumn("dim_model_key", max_value + monotonically_increasing_id())
df_filter_new.display()

1


org.apache.spark.SparkException: [FAILED_READ_FILE.FILE_NOT_EXIST] Error while reading file abfss:REDACTED_LOCAL_PART@carrishitdatalake.dfs.core.windows.net/carSales/part-00000-tid-6548517929100723587-045baea5-5e5e-4a9d-a179-7fa8709ef793-6-1-c000.snappy.parquet. File does not exist. It is possible the underlying files have been updated.
You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
If disk cache is stale or the underlying files have been removed, you can invalidate disk cache manually by restarting the cluster. SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistError(QueryExecutionErrors.scala:1094)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logMissingFileNameAndThrow(FileScanRDD.scala:843)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:741)
	at org.apache.spark.util.N

In [0]:
df_final = df_filter_new.union(df_filter_old)

### SCD Type 1 (Upsert)

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.dim_model"):
    delta_tbl = DeltaTable.forPath(spark, "abfss://gold@carrishitdatalake.dfs.core.windows.net/dim_model")
    delta_tbl.alias("trg").merge(df_final.alias("src"), "trg.dim_model_key = src.dim_model_key") \
                        .whenMatchedUpdateAll() \
                        .whenNotMatchedInsertAll() \
                        .execute()
    pass
else:
    df_final.write.format("delta").mode("overwrite").option("path", "abfss://gold@carrishitdatalake.dfs.core.windows.net/dim_model").saveAsTable("cars_catalog.gold.dim_model")

In [0]:
%sql
select * from cars_catalog.gold.dim_model

Model_ID,model_category,dim_model_key
Mah-M167,Mah,1
Che-M47,Che,2
Toy-M205,Toy,3
BMW-M249,BMW,4
Mer-M122,Mer,5
Hon-M215,Hon,6
Nis-M82,Nis,7
Toy-M206,Toy,8
Mar-M139,Mar,9
Ren-M207,Ren,10
